In [1]:
import requests
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
import logging
import time
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'  

In [2]:
logging.basicConfig(level=logging.INFO)
def fetch_forex_data(currency_from, currency_to, amount=1, precision=4):
    endpoint = f"https://api.polygon.io/v1/conversion/{currency_from}/{currency_to}?amount={amount}&precision={precision}&apiKey={api_key}"
    try:
        response = requests.get(endpoint)
        response.raise_for_status()  
        data = response.json()
        logging.info(data)
        if data.get('status') == 'success':
            return data
        else:
            logging.error(f"Error in response: {data}")
            return None
    except requests.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")  
        return None
    except Exception as err:
        logging.error(f"Other error occurred: {err}") 
        return None

In [3]:
import time
def get_forex_data_for_two_hours(currency_from, currency_to, amount, precision):
    end_time = datetime.now() + timedelta(hours=2)  # set end time = 2 hours later
    next_call_time = datetime.now()  # the time of using API next time
    data_points = []
    
    while datetime.now() < end_time:
        if datetime.now() >= next_call_time:
            data = fetch_forex_data(currency_from, currency_to, amount, precision)
            if data:
                # get data from dict
                timestamp = data.get('last', {}).get('timestamp')
                converted = data.get('converted')
                if timestamp and converted:
                    data_points.append({'time': datetime.fromtimestamp(timestamp / 1000.0), 'mean_rate': converted})
                next_call_time += timedelta(minutes=6)
            else:
                logging.error(f"Failed to fetch data at {datetime.now()}")
        time.sleep(10)

    return pd.DataFrame(data_points)

In [6]:
# run model and calculate slope
def calculate_slope(df):
    df['time_elapsed'] = (df['time'] - df['time'].min()).dt.total_seconds() / 60
    X = df[['time_elapsed']]
    y = df['mean_rate']
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0]  # slope
currency_pairs = [('GBP', 'USD'), ('USD', 'JPY')]
for currency_from, currency_to in currency_pairs:
    df = get_forex_data_for_two_hours(currency_from, currency_to, 1, 4)
    if not df.empty:
        slope = calculate_slope(df)
        print(f"The slope for {currency_from}/{currency_to} is: {slope}")
    else:
        print(f"No data fetched for {currency_from}/{currency_to}.")

INFO:root:{'converted': 1.2549, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.255, 'bid': 1.25491, 'exchange': 48, 'timestamp': 1714716515000}, 'request_id': '8fa4b0fc9f09a8fc9dc773a43f881430', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2549, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.255, 'bid': 1.25491, 'exchange': 48, 'timestamp': 1714716875000}, 'request_id': 'b9388acc40631ab65edf469b8dad96c8', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2546, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.25468, 'bid': 1.25464, 'exchange': 48, 'timestamp': 1714717235000}, 'request_id': '5ef30ba85a04646db141e1b77c252101', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2547, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.2547, 'bid': 1.25466, 'exchange': 48, 'timestamp': 1714717596000}, 'request_id': '5b79a21813fbb5a25a344d6f06a37ac3', 'status': 'success', 'symbol

The slope for GBP/USD is: -1.0018327843214844e-06


INFO:root:{'converted': 153.26, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 153.27, 'bid': 153.26, 'exchange': 48, 'timestamp': 1714724079000}, 'request_id': '85ce08f6c8080c046d2b7863b6540a98', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 153.254, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 153.264, 'bid': 153.254, 'exchange': 48, 'timestamp': 1714724442000}, 'request_id': 'ced3aefcc0228709bb0866fe7e75386a', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 153.317, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 153.329, 'bid': 153.317, 'exchange': 48, 'timestamp': 1714724803000}, 'request_id': '4608f0925b9ce009271e8ef006bef294', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 153.231, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 153.239, 'bid': 153.231, 'exchange': 48, 'timestamp': 1714725163000}, 'request_id': '5f3b997c199d4685b1a76e83f880d61c', 'status': 'success', '

The slope for USD/JPY is: -0.0026485115695229424


In [6]:
# run model and calculate slope
def calculate_slope(df):
    df['time_elapsed'] = (df['time'] - df['time'].min()).dt.total_seconds() / 60
    X = df[['time_elapsed']]
    y = df['mean_rate']
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0]  # slope
def main():
    currency_pairs = [('GBP', 'USD'), ('USD', 'JPY')]
    slopes = {}
    
    for currency_from, currency_to in currency_pairs:
        df = get_forex_data_for_two_hours(currency_from, currency_to, 1, 4)
        if not df.empty:
            slope = calculate_slope(df)
            slopes[f'{currency_from}{currency_to}'] = slope
            print(f"The slope for {currency_from}/{currency_to} is: {slope}")
        else:
            print(f"No data fetched for {currency_from}/{currency_to}.")
        
    if slopes:
        # compare slope
        long_pair = max(slopes, key=slopes.get)
        short_pair = min(slopes, key=slopes.get)
        print(f"Long {long_pair}, Short {short_pair}")

if __name__ == "__main__":
    main()

INFO:root:{'converted': 1.2546, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.25472, 'bid': 1.25463, 'exchange': 48, 'timestamp': 1714752563000}, 'request_id': '1b85b341b0b4176999f26be4d918aed7', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2544, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.25451, 'bid': 1.25442, 'exchange': 48, 'timestamp': 1714752923000}, 'request_id': '097cd46039f9b2c40f47b28751123175', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2539, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.25398, 'bid': 1.25393, 'exchange': 48, 'timestamp': 1714753283000}, 'request_id': '553d61417aa57810be61c600f6d0639e', 'status': 'success', 'symbol': 'GBP/USD', 'to': 'USD'}
INFO:root:{'converted': 1.2545, 'from': 'GBP', 'initialAmount': 1, 'last': {'ask': 1.25458, 'bid': 1.25452, 'exchange': 48, 'timestamp': 1714753644000}, 'request_id': 'ee212d2de6aaa55cc1cf963a503e6a01', 'status': 'success', 's

The slope for GBP/USD is: 2.1413982451051424e-06


INFO:root:{'converted': 152.923, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 152.933, 'bid': 152.923, 'exchange': 48, 'timestamp': 1714760132000}, 'request_id': 'ef42ae7803cb62681c2d0a2424d86694', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 152.948, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 152.958, 'bid': 152.948, 'exchange': 48, 'timestamp': 1714760492000}, 'request_id': 'dfa7d1b521340323275ce11513b69213', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 152.953, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 152.963, 'bid': 152.953, 'exchange': 48, 'timestamp': 1714760851000}, 'request_id': 'ac666cae9066aed2d50ead35d280fdb6', 'status': 'success', 'symbol': 'USD/JPY', 'to': 'JPY'}
INFO:root:{'converted': 152.941, 'from': 'USD', 'initialAmount': 1, 'last': {'ask': 152.954, 'bid': 152.941, 'exchange': 48, 'timestamp': 1714761212000}, 'request_id': '66637913b2d087ee6e83848325b83746', 'status': 'success'

The slope for USD/JPY is: -0.0007003552240410464
Long GBPUSD, Short USDJPY
